In [1]:
# !pip install --upgrade pip
# !pip install pandas
# !pip install sodapy

# Data Profiling

Analyzing a subset of data to understand its characteristics, guiding the design of an efficient data pipeline.

## Data Extraction

In [2]:
import pandas as pd
from sodapy import Socrata
import json
from datetime import datetime, timedelta

In [3]:
with open('config/socrata_config.json') as f:
  config = json.load(f)

AppToken = config['app_token']
UserName = config['user_name']
Password = config["password"]

In [4]:
client = Socrata("data.iowa.gov",
                 AppToken,
                 username = UserName,
                 password = Password,
                 timeout=30)

In [5]:
col_selected = 'invoice_line_no, date, store, name, address, city, zipcode, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, bottle_volume_ml, state_bottle_cost, state_bottle_retail, sale_bottles'
col_list = ['invoice_line_no', 'date', 'store', 'name', 'address', 'city', 'zipcode', 'county', 'category', 'category_name', 'vendor_no', 'vendor_name','itemno', 'im_desc', 'bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail', 'sale_bottles']
len(col_list)

18

In [8]:
# Using data in FY 2023 as a sample
start_date = "2022-07-01"
end_date = "2023-07-01" 

total_rows = []  
limit = 5000  
offset = 0  
more_data = True  

while more_data:
    results = client.get("m3tr-qhgy",
                         select=col_selected, 
                         where=f"(LOWER(name) LIKE '%hy-vee%' OR name LIKE '%WALL TO WALL WINE AND SPIRITS%') AND date >= '{start_date}' AND date < '{end_date}'", 
                         limit=limit, 
                         offset=offset)
    total_rows.extend(results)  

    if len(results) < limit:
        more_data = False
    else:
        offset += limit 

df = pd.DataFrame.from_records(total_rows)
print(f"Rows from {start_date} to {end_date}: {df.shape[0]}")

Rows from 2022-07-01 to 2023-07-01: 749013


In [9]:
# df = df.sample(frac=0.1)
df.head()

,invoice_line_no,date,store,name,address,city,zipcode,county,category,category_name,vendor_no,vendor_name,itemno,im_desc,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles
0,INV-48852600235,2022-07-01T00:00:00.000,2560,HY-VEE FOOD STORE / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302,LINN,1031100,AMERICAN VODKAS,421,SAZERAC COMPANY INC,36900,BARTON NATURALS,1750,6.92,10.38,6
1,INV-48852600108,2022-07-01T00:00:00.000,2560,HY-VEE FOOD STORE / MARION,3600 BUSINESS HWY 151 EAST,MARION,52302,LINN,1011200,STRAIGHT BOURBON WHISKIES,619,CAMPARI AMERICA,22121,WILD TURKEY 81,750,12.00,18.00,6
2,INV-48871900029,2022-07-01T00:00:00.000,2566,HY-VEE FOOD STORE / KNOXVILLE,813 N LINCOLN STE 1,KNOXVILLE,50138,MARION,1011100,BLENDED WHISKIES,297,LAIRD & COMPANY,23828,FIVE STAR,1750,8.25,12.38,6
3,INV-48871900045,2022-07-01T00:00:00.000,2566,HY-VEE FOOD STORE / KNOXVILLE,813 N LINCOLN STE 1,KNOXVILLE,50138,MARION,1081400,AMERICAN SCHNAPPS,434,LUXCO INC,80571,ARROW PEPPERMINT SCHNAPPS MINI,50,4.55,6.83,3
4,INV-48871900044,2022-07-01T00:00:00.000,2566,HY-VEE FOOD STORE / KNOXVILLE,813 N LINCOLN STE 1,KNOXVILLE,50138,MARION,1031100,AMERICAN VODKAS,434,LUXCO INC,36301,HAWKEYE VODKA MINI,50,4.13,6.20,3


## Data Transformation

### 1.1. Removing Duplicate Rows

In [10]:
num_dup = df.duplicated().sum()
perc_dup = num_dup / df.shape[0] * 100 
print(f'Number of duplicate rows: {num_dup}; Percentage of duplicate rows: {perc_dup:.2f}%')

Number of duplicate rows: 2655; Percentage of duplicate rows: 0.35%


In [11]:
# Despite the potential for duplicates arising from reasons other than data entry errors, 
# I'm opting to remove them as they constitute a minimal proportion of the dataset (only 0.02%). 
# This decision is based on the assumption that the impact on data integrity will be negligible while simplifying further analysis.

df = df.drop_duplicates()
num_dup = df.duplicated().sum()
perc_dup = num_dup / df.shape[0] * 100 
print(f'Number of duplicate rows: {num_dup}; Percentage of duplicate rows: {perc_dup:.2f}%')

Number of duplicate rows: 0; Percentage of duplicate rows: 0.00%


In [12]:
# Check if invoice_line_no unique
number_of_duplicates = df.duplicated(subset='invoice_line_no', keep=False).sum()
print(f"Number of duplicates in 'invoice_line_no': {number_of_duplicates}")


Number of duplicates in 'invoice_line_no': 0


### 1.2. Handelling Missing Values

In [13]:
def missing_values_table(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input is not a pandas DataFrame")

    num_na = df.isnull().sum()
    perc_na = (num_na / len(df)) * 100

    df_na = pd.DataFrame({
        'Missing Values': num_na,
        '% of Total Values': perc_na
    })

    df_na = df_na[df_na['Missing Values'] != 0]

    return df_na

missing_values_table(df)

,Missing Values,% of Total Values
address,1,0.000134
city,1,0.000134
zipcode,1,0.000134
county,1,0.000134


Although the current sample shows few missing values, they may be underrepresented in the full dataset. 

These rows, containing key sales data, are vital for our analysis. 
Hence, we will retain them by imputing missing values with placeholders to preserve critical insights. 

However, records lacking essential data such as price, sales volume, dates, and index will be excluded, 
as their absence renders them unsuitable for meaningful analysis.

In [14]:
df.columns

Index(['invoice_line_no', 'date', 'store', 'name', 'address', 'city',
       'zipcode', 'county', 'category', 'category_name', 'vendor_no',
       'vendor_name', 'itemno', 'im_desc', 'bottle_volume_ml',
       'state_bottle_cost', 'state_bottle_retail', 'sale_bottles'],
      dtype='object')

In [17]:
def generate_placeholder_dict(df, cols):
    placeholder_dict = {}
    for col in cols:
        try:
            int_col = df[col].dropna().astype(int)
            mode_digit_count = int_col.astype(str).apply(len).mode()[0]
            placeholder_dict[col] = int('9' * mode_digit_count)
        except ValueError:
            placeholder_dict[col] = 'UNKNOWN'
    return placeholder_dict

dimension_cols = ['store', 'name', 'address', 'city', 'zipcode', 'county', 'category', 'category_name', 'vendor_no', 'vendor_name', 'itemno', 'im_desc']
placeholders = generate_placeholder_dict(df, dimension_cols)

placeholders

{'store': 9999,
 'name': 'UNKNOWN',
 'address': 'UNKNOWN',
 'city': 'UNKNOWN',
 'zipcode': 99999,
 'county': 'UNKNOWN',
 'category': 9999999,
 'category_name': 'UNKNOWN',
 'vendor_no': 999,
 'vendor_name': 'UNKNOWN',
 'itemno': 99999,
 'im_desc': 'UNKNOWN'}

In [18]:
# Save dictionary to a JSON file
with open('dicts/placeholders.json', 'w') as file:
    json.dump(placeholders, file)

In [19]:
df = df.fillna(value=placeholders)
missing_values_table(df)

,Missing Values,% of Total Values


### 2.3. Checking Data Type

Verify if specific columns, such as indexes, are convertible to integers and if the date column can be formatted as datetime for compatibility with corresponding MySQL server data types

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 746358 entries, 0 to 749012
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   invoice_line_no      746358 non-null  object
 1   date                 746358 non-null  object
 2   store                746358 non-null  object
 3   name                 746358 non-null  object
 4   address              746358 non-null  object
 5   city                 746358 non-null  object
 6   zipcode              746358 non-null  object
 7   county               746358 non-null  object
 8   category             746358 non-null  object
 9   category_name        746358 non-null  object
 10  vendor_no            746358 non-null  object
 11  vendor_name          746358 non-null  object
 12  itemno               746358 non-null  object
 13  im_desc              746358 non-null  object
 14  bottle_volume_ml     746358 non-null  object
 15  state_bottle_cost    746358 non-null  o

#### 2.3.1. Datetime Columns

In [21]:
try:
    pd.to_datetime(df['date'])
    print("The 'date' column is in a suitable format for SQL date type.")
except (ValueError, TypeError):
    print("The 'date' column is NOT in a suitable format for SQL date type.")

The 'date' column is in a suitable format for SQL date type.


#### 2.3.2. Integer Columns

In [23]:
num_col_dtype_map = {}

for col in df.columns:
    if df[col].dtype == 'object' and col != 'date':
        # Try to convert the column to numeric type with errors coerced to NaN
        converted_col = pd.to_numeric(df[col], errors='coerce')

        # Check if the column can be converted to a numeric type
        if not converted_col.isna().any():
            # If all numeric values are integers, categorize as 'int', otherwise 'float'
            if (converted_col % 1 == 0).all():
                num_col_dtype_map[col] = 'int'
                print(f"The column '{col}' can be fully converted to integers.")
            else:
                num_col_dtype_map[col] = 'float'
                print(f"The column '{col}' can be fully converted to floats.")
        else:
            print(f"The column '{col}' CANNOT be fully converted to numbers.")

print(num_col_dtype_map)

The column 'invoice_line_no' CANNOT be fully converted to numbers.
The column 'store' can be fully converted to integers.
The column 'name' CANNOT be fully converted to numbers.
The column 'address' CANNOT be fully converted to numbers.
The column 'city' CANNOT be fully converted to numbers.
The column 'zipcode' can be fully converted to integers.
The column 'county' CANNOT be fully converted to numbers.
The column 'category' can be fully converted to integers.
The column 'category_name' CANNOT be fully converted to numbers.
The column 'vendor_no' can be fully converted to integers.
The column 'vendor_name' CANNOT be fully converted to numbers.
The column 'itemno' can be fully converted to integers.
The column 'im_desc' CANNOT be fully converted to numbers.
The column 'bottle_volume_ml' can be fully converted to integers.
The column 'state_bottle_cost' can be fully converted to floats.
The column 'state_bottle_retail' can be fully converted to floats.
The column 'sale_bottles' can be f

In [24]:
# Save dictionary to a JSON file
with open('dicts/num_col_dtype_map.json', 'w') as file:
    json.dump(num_col_dtype_map, file)

### 2.4. Data Validation

In [26]:
def convert_to_numeric(df, col):
    # Convert column to float first
    df[col] = df[col].astype(float)
    # If the float value is equal to its integer conversion, change to int
    df[col] = df[col].apply(lambda x: int(x) if x == int(x) else x)

columns_to_convert = ['bottle_volume_ml', 'state_bottle_cost', 'state_bottle_retail', 'sale_bottles']
for col in columns_to_convert:
    convert_to_numeric(df, col)

In [27]:
df.describe()

,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles
count,746358.000000,746358.000000,746358.000000,746358.000000
mean,869.723310,13.120364,19.679969,11.892889
std,512.980053,26.088963,39.132057,43.917080
min,20.000000,0.890000,1.340000,-360.000000
25%,750.000000,7.000000,10.500000,3.000000
50%,750.000000,10.000000,15.000000,6.000000
75%,1000.000000,15.910000,23.870000,12.000000
max,5250.000000,18436.000000,27654.000000,11880.000000


In [28]:
# Check the percentage of rows where 'sale_bottles' is negative
len(df[df['sale_bottles']<0])/len(df) * 100

0.0896352688656114

**Note**: While negative sales data typically indicate refunds or returns, our analysis will omit these records to maintain data integrity. 

This decision is based on the focus of our study, which requires consistent and positive sales figures to accurately assess trends and patterns.


In [29]:
df=df[df['sale_bottles']>=0]
df.describe()

,bottle_volume_ml,state_bottle_cost,state_bottle_retail,sale_bottles
count,745689.000000,745689.000000,745689.000000,745689.000000
mean,869.726655,13.115803,19.673125,11.913347
std,513.007834,25.968483,38.951329,43.927132
min,20.000000,0.890000,1.340000,1.000000
25%,750.000000,7.000000,10.500000,3.000000
50%,750.000000,10.000000,15.000000,6.000000
75%,1000.000000,15.910000,23.870000,12.000000
max,5250.000000,18436.000000,27654.000000,11880.000000
